# Sensitivity test: full Maxey-Riley equation vs slow manifold MR equation
creation date: 19/06/2025  
author: Meike Bos  
Description: In this notebook we compare the trajectories of particles advected with the full Maxey-Riley(MR) equation vs paticles advected with the slow manifold (SM) MR equation  
We do not include the history term (not possible for the SM MR eq). We compare simulations with a constant Rep = 0, 450 and a flexible Rep.   

Comparing will be
- separation distance 

29/08/2025 update: removed displaced tracer from comparison



In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

# import needed packages
import sys
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import Divider, Size
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta




sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av, skill_score
from analysis_functions import make_PDF, Haversine, make_lognormal_PDF
from particle_characteristics_functions import factor_drag_white1991, factor_drag_Schiller1933, factor_drag_morrison2013, Re_particle, stokes_relaxation_time

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')


Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.9, 5))

In [ ]:
# filenames and inpput settings
Replist = [0,10,100,450,1000] 
coriolis = True
gradient = True
B = 0.68
tau = 3196 #2994.76 #2759.97
runtime =  timedelta(days=2)# timedelta(days=10)
land_handling = 'anti_beaching'
nparticles = 88347 # 52511
chunck_time = 100
loc = 'NWES'
displacement = 300 # m 

starttimes = [datetime(2023, 9, 1, 0, 0, 0, 0)]


base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'
basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_tracer  = (base_directory + '{particle_type}/{loc}_'
                        'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                        'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')

basefile_tracer_random = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')

basefiles={'tracer':basefile_tracer,
           'tracer_random':basefile_tracer_random,
           'inertial_Rep_constant':basefile_Rep_constant,
           'inertial_SM_Rep_constant':basefile_Rep_constant, 
           'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_Rep_drag}

particle_types = ['tracer','inertial_SM_drag_Rep','inertial_drag_Rep','inertial_SM_Rep_constant','inertial_Rep_constant']#,'inertial_Rep_constant'] #  # 'tracer_random'
names = {'tracer':'tracer',
         'tracer_random':'displaced tracer',
         'inertial_Rep_constant':'MR',
           'inertial_SM_Rep_constant':'SM MR', 
           'inertial_SM_drag_Rep':'SM MR flexible Re$_p$',
           'inertial_drag_Rep':'MR flexible Re$_p$'}


In [ ]:
# read in data
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in [True]:
        data[pt][coriolis]={}
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
            for Rep in Replist:
                data[pt][coriolis][Rep]={}
        else:
            data[pt][coriolis][None]={}



for pt in particle_types:
    for coriolis in [True]:
        for starttime in starttimes:
            # print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            
            if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
                for Rep in Replist:
                    file = basefiles[pt].format(loc=loc,
                                                y_s=starttime.year,
                                                m_s=starttime.month,
                                                d_s=starttime.day,
                                                y_e=endtime.year,
                                                m_e=endtime.month,
                                                d_e=endtime.day,
                                                B = int(B * 1000), 
                                                tau = int(tau ),
                                                land_handling = land_handling, 
                                                coriolis = coriolis,
                                                gradient = gradient,
                                                particle_type = pt,
                                                Rep = Rep)
                    ds = xr.open_dataset(file,
                                        engine='zarr',
                                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                                        drop_variables=['B','tau','z'],
                                        decode_times=True) #,decode_cf=False)

                    data[pt][coriolis][Rep][date]= ds 
            else:
                print (pt)
                file = basefiles[pt].format(loc=loc,
                                            y_s=starttime.year,
                                            m_s=starttime.month,
                                            d_s=starttime.day,
                                            y_e=endtime.year,
                                            m_e=endtime.month,
                                            d_e=endtime.day,
                                            land_handling = land_handling, 
                                            coriolis = coriolis,
                                            gradient = gradient,
                                            d=displacement,
                                            B = int(B * 1000), 
                                            tau = int(tau ),
                                            particle_type = pt)
                print(file)
                ds = xr.open_dataset(file,
                                    engine='zarr',
                                    chunks={'trajectory':nparticles, 'obs':chunck_time},
                                    drop_variables=['z'],
                                    decode_times=True) #,decode_cf=False)

                data[pt][coriolis][None][date]= ds 



In [ ]:
# plot selection of trajectories
Blues = colormaps['Blues']

fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1),projection=ccrs.PlateCarree())
# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.95, 5))
colorlist=['k','grey','cornflowerblue','blue']

year=2023
month=9
date = f'{year:04d}/{month:02d}'
legend= []
coriolis = True

ax.coastlines()
idstart =25#25 #600#100#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
idstep = 1
idend = idstart+1 * idstep
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')
markers=['-','--','-','--','-','--']
for pt, marker in zip(particle_types,markers):
        print(pt)
        # pt = 'inertial_SM_Rep_constant'
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
                for Rep, color2 in zip(Replist,Magmalist):
                        ax.plot(data[pt][coriolis][Rep][date].lon[idstart:idend:idstep,0::1].T,
                                data[pt][coriolis][Rep][date].lat[idstart:idend:idstep,0::1].T,
                                marker,
                                color=color2);

                        legend.append(names[pt]+' Re$_p$={:d}'.format(Rep)) #simtype[pt]+
        elif(pt in ('tracer','tracer_random')):
                ax.plot(data[pt][coriolis][None][date].lon[idstart,0::1].T,
                data[pt][coriolis][None][date].lat[idstart,0::1].T,
                marker,
                color='k',zorder=20);
                legend.append(names[pt])
        else:
                ax.plot(data[pt][coriolis][None][date].lon[idstart,0::1].T,
                data[pt][coriolis][None][date].lat[idstart,0::1].T,
                marker,
                color='cornflowerblue',zorder=20);
                legend.append(names[pt])
ax.legend(legend,fontsize=18, handlelength = 1.5,loc=(1,0.))
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}

In [ ]:
# add tidal average lons and lats 
window=25
startnan=np.full((nparticles,25),np.nan)
endnan=np.full((nparticles,80),np.nan)
Tend=720
coriolis=True
for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'

    for pt in particle_types:
        print(pt)
        
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
             for Rep in Replist:
                da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][Rep][date].lon,window)
                da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][Rep][date].lat,window)

                data[pt][coriolis][Rep][date] = data[pt][coriolis][Rep][date].assign(lon_tidal_av = da_lon_tidal_av)
                data[pt][coriolis][Rep][date] = data[pt][coriolis][Rep][date].assign(lat_tidal_av = da_lat_tidal_av)
            
        else:
            da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][None][date].lon,window)
            da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][None][date].lat,window)

            data[pt][coriolis][None][date] = data[pt][coriolis][None][date].assign(lon_tidal_av = da_lon_tidal_av)
            data[pt][coriolis][None][date] = data[pt][coriolis][None][date].assign(lat_tidal_av = da_lat_tidal_av)
           

In [ ]:
# along track distance
for pt in particle_types: # loop over all particle types
    for starttime in starttimes: # loop over all starttime
        year = starttime.year
        month = starttime.month
        date = f'{year:04d}/{month:02d}'
        if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
            for Rep in Replist:
                ds =data[pt][coriolis][Rep][date]
                traj_length = trajectory_length(ds.lon, ds.lat)
                ds = ds.assign(traj_length = traj_length)
                # traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:], ds.lat_tidal_av[:,window:])
                # ds = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
                data[pt][coriolis][Rep][date] = ds
    
        else:
            ds=data[pt][coriolis][None][date]
            traj_length = trajectory_length(ds.lon, ds.lat)
            ds = ds.assign(traj_length = traj_length)
            # traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:],ds.lat_tidal_av[:,window:])
            # ds = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
            data[pt][coriolis][None][date] = ds
            


In [ ]:
#  plot PDF of total trajectory length
fig,ax=plt.subplots()

Tmax = 48*12

legend=[]
# Tmax=500
for pt, marker in zip(particle_types,markers):




    if(pt in ('inertial_SM_Rep_constant','inertial_Rep_constant')):
        for Rep, color2 in zip(Replist,Magmalist[::2]): 
            legend.append(names[pt]+' Re$_p^{\\mathrm{In}}$ ='+' {:d}'.format(Rep))
            data_arrays = list(data[pt][coriolis][Rep].values())
            combined_data = xr.concat(data_arrays, dim="starting_days")

            average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),101, norm=True,vmin=0,vmax=600)
            ax.plot(bins,pdf,marker,color=color2)
    elif(pt in ('tracer','tracer_random')):
        legend.append(names[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")

        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),101, norm=True,vmin=0,vmax=600)


        ax.plot(bins,pdf,marker,color='k',zorder=20)
    else:
        legend.append(names[pt])
        data_arrays = list(data[pt][coriolis][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days")

        average_data = combined_data.traj_length.mean(dim="starting_days",skipna=True)

        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),101, norm=True,vmin=0,vmax=600)


        ax.plot(bins,pdf,marker,color='cornflowerblue',zorder=20)
# ax.set_title(simtype[pt])
ax.legend(legend,fontsize=18)
ax.set_ylabel("PDF")
ax.set_xlabel("trajectory length (km)")


    #tidal_av
   


# for particle_type in particle_types:
#     bins, pdf = make_PDF(arrays[particle_type+'_tidal_av'],401, norm=True)
#     ax.plot(bins,pdf,'--',color=color_particles[particle_type])
#     legend.append(particle_type+' tidal av')



In [ ]:
# Relative dispersion
pt_pairs = [['inertial_Rep_constant','inertial_SM_Rep_constant'], 
            ['inertial_drag_Rep','inertial_SM_drag_Rep'],
            ['inertial_Rep_constant','tracer'],
            ['inertial_drag_Rep','tracer'],
            ['inertial_SM_Rep_constant','tracer'],
            ['inertial_SM_drag_Rep','tracer']]


for starttime in starttimes:
    print(starttime)
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'

    for pt1, pt2 in pt_pairs:
        if(pt1 == 'inertial_Rep_constant'):
            for Rep in Replist:
                ds_Rep_MR = data[pt1][coriolis][Rep][date]
                if(pt2 == 'inertial_SM_Rep_constant'):
                    ds_compare = data[pt2][coriolis][Rep][date]
                    name_dist= 'dist_to_SM'
                else:
                    ds_compare = data[pt2][coriolis][None][date]
                    name_dist= 'dist_to_tracer'
               
                dist= Haversine(ds_Rep_MR['lon'],ds_Rep_MR['lat'],
                                    ds_compare['lon'],ds_compare['lat'])
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].assign(dist = dist)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].rename({'dist':name_dist})

        elif(pt1 == 'inertial_SM_Rep_constant'):
            ds_compare = data[pt2][coriolis][None][date]
            name_dist= 'dist_to_tracer'
            for Rep in Replist:
                ds_Rep_MR = data[pt1][coriolis][Rep][date]
                dist= Haversine(ds_Rep_MR['lon'],ds_Rep_MR['lat'],
                                    ds_compare['lon'],ds_compare['lat'])
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].assign(dist = dist)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].rename({'dist':name_dist})
        elif(pt1 == 'inertial_SM_drag_Rep'):
            ds_compare = data[pt2][coriolis][None][date]
            ds_Rep_MR = data[pt1][coriolis][None][date]
            name_dist= 'dist_to_tracer'
            dist= Haversine(ds_Rep_MR['lon'],ds_Rep_MR['lat'],
                                    ds_compare['lon'],ds_compare['lat'])
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(dist = dist)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'dist':name_dist})


        else:
            ds_MR = data[pt1][coriolis][None][date]
            if(pt2 == 'inertial_SM_drag_Rep'):
                ds_compare = data[pt2][coriolis][None][date]
                name_dist= 'dist_to_SM'
            else:
                ds_compare = data[pt2][coriolis][None][date]
                name_dist= 'dist_to_tracer'
               
            dist= Haversine(ds_MR['lon'],ds_MR['lat'],
                                    ds_compare['lon'],ds_compare['lat'])
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(dist = dist)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'dist':name_dist})
            data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].assign(dist = dist)
            data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].rename({'dist':'dist_MR_full'})

            # #tial av
            # dist_tidal_av= Haversine(ds_MR['lon_tidal_av'],ds_MR['lat_tidal_av'],
            #                         ds_SMMR['lon_tidal_av'],ds_SMMR['lat_tidal_av'])
            # data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(dist_tidal_av = dist_tidal_av)
            # data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'dist_tidal_av':'dist_MR_full_tidal_av'})
            # data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].assign(dist_tidal_av = dist_tidal_av)
            # data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].rename({'dist_tidal_av':'dist_MR_full_tidal_av'})



In [ ]:
#check that particles do not shoot off anymore
ds = data['inertial_drag_Rep'][coriolis][None][date]
mask = ds['lat'] < 45

# Find which trajectories have any observation with lon > 60
selected_trajectories = mask.any(dim='obs')


# Use the mask to select those trajectories from the orig
ds_selected = ds.sel(trajectory=selected_trajectories)

# no shooting off particles (trajectorys is 0)
print(ds_selected)

In [ ]:
# relative distance between tracer and MRG particles is much larger than relative distance between MRG and SM-MRG particles ()
Tmax=48*12
tlist=np.arange(1,Tmax-1,1)/24/12
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))#,projection=ccrs.PlateCarree())
legend=[]

for pt1 in ['inertial_Rep_constant','inertial_drag_Rep']:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    if(pt1 == 'inertial_Rep_constant'):
            for Rep,color2 in zip(Replist,Magmalist):
                ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color=color2)
                legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to tracer')
                ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_SM'].mean(dim='trajectory',skipna=True)[2:Tmax],'-.',color=color2)
                legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to SM')
    else:
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color='cornflowerblue')
        legend.append('flexible Re$_p$ to tracer')
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_SM'].mean(dim='trajectory',skipna=True)[2:Tmax],'-.',color='cornflowerblue')
        legend.append('flexible Re$_p$ to SM')
        
ax.set_xlabel('time [days]')
ax.set_ylabel('relative distance [km]')

ax.set_yscale('log')

ax.legend(['MRG and tracer','MRG and SM-MRG'])




In [ ]:
# relative distance for MRG-tracer and SM-MRG and tracer

Tmax=48*12
tlist=np.arange(1,Tmax-1,1)/24/12
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))
legend=[]

for pt1 in ['inertial_Rep_constant','inertial_SM_drag_Rep','inertial_SM_Rep_constant','inertial_drag_Rep']:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    if(pt1 == 'inertial_Rep_constant'):
            for Rep,color2 in zip(Replist,Magmalist):
                print(Rep)
                if(Rep == 0):
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color=color2)
                    legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to tracer')
                    print(color2)
                else: 
                     ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color=color2,label='_nolegend_')
    elif(pt1 == 'inertial_SM_Rep_constant'):
            for Rep,color2 in zip(Replist,Magmalist):
                ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'--',color=color2)#,label='_nolegend_')
                legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to tracer')
    elif(pt1 == 'inertial_drag_Rep'):
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color='cornflowerblue',label='_nolegend_')
        legend.append('flexible Re$_p$ to tracer')
    else:
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'--',color='cornflowerblue',label='_nolegend_')
        legend.append('flexible Re$_p$ to tracer')

        

ax.set_xlabel('time [days]')
ax.set_ylabel('relative distance [km]')
ax.legend(['MRG and tracer','SM-MRG and tracer'])

ax.set_yscale('log')
fig.savefig('../figures/MRG_vs_SM-MRG/relative_distance_MRG_tracer.pdf')

In [ ]:
# relative distance MRG-tracer, SM-MRG-tracer and MRG-SM-MRG

Tmax=48*12
tlist=np.arange(1,Tmax-1,1)/24
fig,ax=plt.subplots()
# ax2 = ax.twinx()
legend=[]

for pt1 in ['inertial_Rep_constant','inertial_SM_drag_Rep','inertial_SM_Rep_constant','inertial_drag_Rep']:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    if(pt1 == 'inertial_Rep_constant'):
            for Rep,color2 in zip(Replist,Magmalist):
                print(Rep)
                if(Rep == 0):
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color=color2,label=1)
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_SM'].mean(dim='trajectory',skipna=True)[2:Tmax],':',color=color2,label=3)#,label='_nolegend_')
                    legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to tracer')
                    print(color2)
                else: 
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color=color2,label='_nolegend_')
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_SM'].mean(dim='trajectory',skipna=True)[2:Tmax],':',color=color2,label='_nolegend_')

    elif(pt1 == 'inertial_SM_Rep_constant'):
            for Rep,color2 in zip(Replist,Magmalist):
                if(Rep == 0):
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'--',color=color2,label=2)#,label='_nolegend_')
                    legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to tracer')
                else: 
                    ax.plot(tlist,data[pt1][coriolis][Rep][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'--',color=color2,label='_nolegend_')
                    legend.append('Re$_p^{\\mathrm{In}}$= '+f'{Rep} to tracer')
    elif(pt1 == 'inertial_drag_Rep'):

        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'-',color='cornflowerblue',label='_nolegend_')
        legend.append('flexible Re$_p$ to tracer')
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_SM'].mean(dim='trajectory',skipna=True)[2:Tmax],':',color='cornflowerblue')
        legend.append('flexible Re$_p$ to SM')
    else:
        ax.plot(tlist,data[pt1][coriolis][None][date]['dist_to_tracer'].mean(dim='trajectory',skipna=True)[2:Tmax],'--',color='cornflowerblue',label='_nolegend_')
        legend.append('flexible Re$_p$ to tracer')

ax.set_xlabel('time [days]')
ax.set_ylabel('relative distance[km]')

handles, labels = ax.get_legend_handles_labels()
# sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles, ['MRG and tracer','SM-MRG and tracer','MRG and SM-MRG'])

ax.set_yscale('log')


In [ ]:
# skill score
pt_pairs = [['inertial_Rep_constant','inertial_SM_Rep_constant'], 
            ['inertial_drag_Rep','inertial_SM_drag_Rep'],
            ['inertial_Rep_constant','tracer'],
            ['inertial_drag_Rep','tracer'],
            ['inertial_SM_Rep_constant','tracer'],
            ['inertial_SM_drag_Rep','tracer']]


for starttime in starttimes:
    print(starttime)
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}' #'inertial_SM_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']

    for pt1, pt2 in pt_pairs:
        if(pt1 == 'inertial_Rep_constant'):
            for Rep in Replist:
                ds_Rep_MR = data[pt1][coriolis][Rep][date]
                if(pt2 == 'inertial_SM_Rep_constant'):
                    ds_compare = data[pt2][coriolis][Rep][date]
                    name_ss= 'ss_to_SM'
                else:
                    ds_compare = data[pt2][coriolis][None][date]
                    name_ss= 'ss_to_tracer'
               
                ss= skill_score(ds_Rep_MR.lon,ds_Rep_MR.lat,ds_compare.lon,ds_compare.lat)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].assign(ss = ss)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].rename({'ss':name_ss})


        elif(pt1 == 'inertial_SM_Rep_constant'):
            ds_compare = data[pt2][coriolis][None][date]
            name_ss= 'ss_to_SM'
            for Rep in Replist:
                ds_Rep_MR = data[pt1][coriolis][Rep][date]
                ss= skill_score(ds_Rep_MR.lon,ds_Rep_MR.lat,ds_compare.lon,ds_compare.lat)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].assign(ss = ss)
                data[pt1][coriolis][Rep][date]=data[pt1][coriolis][Rep][date].rename({'ss':name_ss})
        elif(pt1 == 'inertial_SM_drag_Rep'):
            ds_compare = data[pt2][coriolis][None][date]
            ds_Rep_MR = data[pt1][coriolis][None][date]
            name_ss= 'ss_to_tracer'
            ss= skill_score(ds_Rep_MR.lon,ds_Rep_MR.lat,ds_compare.lon,ds_compare.lat)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(ss = ss)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'ss':name_ss})


        else:
            ds_MR = data[pt1][coriolis][None][date]
            if(pt2 == 'inertial_SM_drag_Rep'):
                ds_compare = data[pt2][coriolis][None][date]
                name_ss= 'ss_to_SM'
            else:
                ds_compare = data[pt2][coriolis][None][date]
                name_ss= 'ss_to_tracer'
               
            ss= skill_score(ds_MR.lon,ds_MR.lat,ds_compare.lon,ds_compare.lat)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].assign(ss = ss)
            data[pt1][coriolis][None][date]=data[pt1][coriolis][None][date].rename({'ss':name_ss})
            # data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].assign(ss = ss)
            # data[pt2][coriolis][None][date]=data[pt2][coriolis][None][date].rename({'ss':name_ss})

   



In [ ]:
# skill score ovr time (it converges to constant value ovre time)
fig,ax= plt.subplots()
legend=[]
for Rep, color in zip(Replist,Magmalist):
    ax.plot(data['inertial_Rep_constant'][coriolis][Rep][date].ss_to_SM.mean(dim='trajectory',skipna=True)[12:Tmax],'-.',color=color)
    legend.append(f'c({Rep:.0f})')
ax.plot(data['inertial_drag_Rep'][coriolis][None][date].ss_to_SM.mean(dim='trajectory',skipna=True)[12:Tmax],'--',color='cornflowerblue')
legend.append(f'c$_p$(t)')
ax.set_xlabel('time [hours]')
ax.set_ylabel('skillscore')
ax.legend(legend,fontsize=18)

In [ ]:
fig, ax = plt.subplots()
ax.plot(data['tracer'][coriolis][None][date]['lon'].isel(trajectory=100,obs=slice(0,48)).values)
ax.plot(data['inertial_SM_drag_Rep'][coriolis][None][date]['lon'].isel(trajectory=100,obs=slice(0,48)).values)
ax.plot(data['inertial_drag_Rep'][coriolis][None][date]['lon'].isel(trajectory=100,obs=slice(0,48)).values)
#-data['tracer'][coriolis][None][date]['lon'].isel(trajectory=1).values

In [ ]:
Tmax = 48*12
fig = plt.figure()
h = [Size.Fixed(1.0), Size.Fixed(8)]
v = [Size.Fixed(0.7), Size.Fixed(6)]
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
ax = fig.add_axes(divider.get_position(),
                  axes_locator=divider.new_locator(nx=1, ny=1))

i=1
for Rep, color in zip(Replist,Magmalist):
    error = np.array([(data['inertial_Rep_constant'][coriolis][Rep][date].ss_to_SM).std(dim='trajectory',skipna=True)[Tmax-1].values,0])
    ax.errorbar(i,(data['inertial_Rep_constant'][coriolis][Rep][date].ss_to_SM).mean(dim='trajectory',skipna=True)[Tmax-1],np.array([error]).T,fmt='o',color=color,markersize=12)
    i+=1

error = np.array([(data['inertial_drag_Rep'][coriolis][None][date].ss_to_SM).std(dim='trajectory',skipna=True)[Tmax-1].values,0])
ax.errorbar(i,(data['inertial_drag_Rep'][coriolis][None][date].ss_to_SM).mean(dim='trajectory',skipna=True)[Tmax-1],np.array([error]).T,fmt='s',color='cornflowerblue',markersize=12)



ax.set_xticks([1,2,3,4,5,6],['c(0)','c(10)','c(100)','c(450)','c(1000)','c$_p$(t)'])

ax.set_ylabel('skill score')
fig.savefig('../figures/MRG_vs_SM-MRG/skill_score_Rep.pdf')